# MULTICAST

Apartado 1

In [1]:
import socket
import struct

MCAST_GRP = '224.1.1.1'
MCAST_PORT = 5007

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

#Usar '' para esuchar y escribir en el socket
#Usar MCAST_GRP para escuchar solo en el socket
sock.bind(('', MCAST_PORT))
sock.sendto(str.encode("Hello this is user 1"), (MCAST_GRP, MCAST_PORT))

mreq = struct.pack("4sl", socket.inet_aton(MCAST_GRP), socket.INADDR_ANY)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, mreq)

#Establecer el ttl
ttl = struct.pack('b',1)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_MULTICAST_TTL, ttl)

msg = ""
while str(msg) != "Adios":
    mensaje_tx = str.encode(input('Introduzca el mensaje a enviar: '))
    sock.sendto(mensaje_tx, (MCAST_GRP, MCAST_PORT))
    msg = sock.recv(1024).decode()
    print("RX: ", msg)
    
print("Cliente abandona el grupo")                       
sock.close()

Introduzca el mensaje a enviar: Hola 
RX:  Hello this is user 2
Introduzca el mensaje a enviar: soy cliente 1
RX:  Hola 


KeyboardInterrupt: Interrupted by user

Apartado 2

In [1]:
import socket
import struct
import pickle
import threading

MCAST_GRP = '224.1.1.1'
MCAST_PORT = 5007

# identificador del participante
user_id = 1

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

#Usar '' para esuchar y escribir en el socket
#Usar MCAST_GRP para escuchar solo en el socket
sock.bind(('', MCAST_PORT))
mensaje_bienvenida = 'Hello this is user '
bytes_bienvenida = pickle.dumps([mensaje_bienvenida, user_id])
sock.sendto(bytes_bienvenida, (MCAST_GRP, MCAST_PORT))

mreq = struct.pack("4sl", socket.inet_aton(MCAST_GRP), socket.INADDR_ANY)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, mreq)

#Establecer el ttl
ttl = struct.pack('b',1)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_MULTICAST_TTL, ttl)

def time_function(name):
    global stop
    while stop == False:
        msg = sock.recv(1024)
        mensaje = pickle.loads(msg)
        print("Thread User " + str(mensaje[1]) + ": " + str(mensaje[0]))
        
    print("Cliente abandona el grupo")
    sock.close()
    
stop = False
reader = threading.Thread(target=time_function, args=(user_id,))
reader.start()

while stop == False:
    mensaje_tx = input('Introduzca el mensaje a enviar: ')
    bytes_tx = pickle.dumps([mensaje_tx, user_id])
    sock.sendto(bytes_tx, (MCAST_GRP, MCAST_PORT))
    if mensaje_tx == "quit":
        stop = True

sock.close()

Thread User 2: Hello this is user 
Thread User 2: aojdas
Introduzca el mensaje a enviar: asod
Thread User 1: asod
Introduzca el mensaje a enviar: quit
Thread User 1: quit
Cliente abandona el grupo
